In [1]:
!rm -rf /kaggle/working/adversarial-patch-transferability
!rm -rf /kaggle/working/adversarial-patch-transferability/sam2
!rm -rf /kaggle/working/adversarial-patch-transferability/sam2/sam2
!rm -rf /kaggle/working/sam2

In [2]:
!git clone -b noiseinit https://github.com/AGAMPANDEYY/adversarial-patch-transferability.git

Cloning into 'adversarial-patch-transferability'...
remote: Enumerating objects: 589, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 589 (delta 72), reused 49 (delta 49), pack-reused 485 (from 2)
Receiving objects: 100% (589/589), 7.32 MiB | 24.99 MiB/s, done.
Resolving deltas: 100% (313/313), done.


In [ ]:
%cd ..
!ls
%cd sam2

In [9]:
%cd adversarial-patch-transferability
!git fetch origin noiseinit
!git pull origin noiseinit

/kaggle/working/adversarial-patch-transferability
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 1.41 KiB | 1.41 MiB/s, done.
From https://github.com/AGAMPANDEYY/adversarial-patch-transferability
 * branch            noiseinit  -> FETCH_HEAD
   53185a2..390030c  noiseinit  -> origin/noiseinit
From https://github.com/AGAMPANDEYY/adversarial-patch-transferability
 * branch            noiseinit  -> FETCH_HEAD
Updating 53185a2..390030c
Fast-forward
 pretrained_models/SegFormer/model.py | 33 ++++++++++++++++++++-------------
 1 file changed, 20 insertions(+), 13 deletions(-)


In [3]:
!git clone https://github.com/facebookresearch/sam2.git

Cloning into 'sam2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 128.11 MiB | 40.09 MiB/s, done.
Resolving deltas: 100% (380/380), done.


In [4]:
%cd sam2

/kaggle/working/sam2


In [5]:
!pip install --q -e .

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!mkdir -p checkpoints/

In [7]:
!ls

assets		    docker-compose.yaml  pyproject.toml    setup.py
backend.Dockerfile  INSTALL.md		 README.md	   tools
checkpoints	    LICENSE		 RELEASE_NOTES.md  training
CODE_OF_CONDUCT.md  LICENSE_cctorch	 sam2
CONTRIBUTING.md     MANIFEST.in		 SAM_2.egg-info
demo		    notebooks		 sav_dataset


In [8]:
!curl -L https://dl.fbaipublicfiles.com/sam2/checkpoints/sam2_hiera_tiny.pt \
      -o checkpoints/sam2_hiera_tiny.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111    0   111    0     0    437      0 --:--:-- --:--:-- --:--:--   438


In [9]:
!pwd

/kaggle/working/sam2


In [10]:
!ls checkpoints

download_ckpts.sh  sam2_hiera_tiny.pt


## Testing on PIDNet-S, PIDNet-M, BiSeNetv2, ICNet

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import gc
sys.path.append('/kaggle/working/adversarial-patch-transferability')
from utils.utils import setup_logging, get_config_from_yaml, process_config, print_config
from dataset.cityscapes import Cityscapes
from metrics.performance import SegmentationMetric
from utils.helper import val_plot
from patch.create import Patch
#from metrics.performance import SegmentationMetric


from pretrained_models.ICNet.icnet import ICNet
from pretrained_models.BisNetV2.model import BiSeNetV2
from pretrained_models.PIDNet.model import PIDNet, get_pred_model
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2

import pickle
from copy import deepcopy
# from trainer.trainer import Trainer
# import torch
from tqdm import tqdm
config = get_config_from_yaml('/kaggle/working/adversarial-patch-transferability/configs/config.yaml')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [12]:
import pickle
pidnet_s_p = pickle.load(open( "/kaggle/input/patch-2/pidnet_s (2).p", "rb" ))[0]

pidnet_s_p = pickle.load(open( "/kaggle/input/noiseinit_kldivergence/pytorch/default/1/pidnet_s.p", "rb" ))[0]
pidnet_s_p = pickle.load(open( "/kaggle/input/ensemble_transeg/pytorch/default/1/pidnet_s (2).p", "rb" ))[0]
patches = {
    'pidnet_s':pidnet_s_p
}

for patch in patches:
  patches[patch] = patches[patch].to(device)
  print(patch)


pidnet_s


## Pytorch optimised patch

In [ ]:
import torch

# Load the .pt file
pidnet_s_p = torch.load("/kaggle/input/patch_greedy/pytorch/default/1/optimized_patch.pt")

# Put patch into a dictionary (if you want to keep the same structure)
patches = {
    'pidnet_s': pidnet_s_p
}

# Move to device
for patch in patches:
    patches[patch] = patches[patch].to(device)
    print(patch)


In [13]:
cityscape_val = Cityscapes(
          root = config.dataset.root,
          list_path = config.dataset.val,
          num_classes = config.dataset.num_classes,
          multi_scale = False,
          flip = False,
          ignore_label = config.train.ignore_label,
          base_size = config.train.base_size,
          crop_size = (config.train.height,config.train.width),
        )

val_dataloader = torch.utils.data.DataLoader(dataset=cityscape_val,
                                            batch_size=1,
                                            shuffle=True,
                                            num_workers=config.train.num_workers,
                                            pin_memory=config.train.pin_memory,
                                            drop_last=config.train.drop_last)

In [19]:
## PIDNet-s
model = torch.load('/kaggle/input/pidnet_s_cityscapes_test/pytorch/default/1/PIDNet_S_Cityscapes_test.pt',map_location=device)
pidnet_s = get_pred_model(name = 'pidnet_s', num_classes = 19).to(device)
if 'state_dict' in model:
    model = model['state_dict']
model_dict = pidnet_s.state_dict()
model = {k[6:]: v for k, v in model.items() # k[6:] to start after model. in key names
                    if k[6:] in model_dict.keys()}

pidnet_s.load_state_dict(model)
pidnet_s.eval()
print('PIDNet-s Model loaded')

## PIDNet-m
model = torch.load('/kaggle/input/pidnet_m_cityscapes_test/pytorch/default/1/PIDNet_M_Cityscapes_test.pt',map_location=device)
pidnet_m = get_pred_model(name = 'pidnet_m', num_classes = 19).to(device)
if 'state_dict' in model:
    model = model['state_dict']
model_dict = pidnet_m.state_dict()
model = {k[6:]: v for k, v in model.items() # k[6:] to start after model. in key names
                    if k[6:] in model_dict.keys()}

pidnet_m.load_state_dict(model)
pidnet_m.eval()
print('PIDNet-m Model loaded')

## PIDNet-l
try:
    model = torch.load('/kaggle/input/pidnet-l-weights/PIDNet_L_Cityscapes_test.pt',
                       map_location=device)
    pidnet_l = get_pred_model(name='pidnet_l', num_classes=19).to(device)
    if 'state_dict' in model:
        model = model['state_dict']
    model_dict = pidnet_l.state_dict()
    model = {k[6:]: v for k, v in model.items() if k[6:] in model_dict.keys()}
    pidnet_l.load_state_dict(model)
    pidnet_l.eval()
    print('PIDNet-L loaded')
except Exception as e:
    pidnet_l = None
    print(f'WARNING: PIDNet-L not loaded: {e}')

# ---- metrics for new models ----
metric_pidnet_l   = SegmentationMetric(config)

# ## ICNet
# model = torch.load('/content/drive/MyDrive/Colab Notebooks/1_Papers/3_Attack_generation/pretrained_models/ICNet/Copy of resnet50_2024-12-22 08:52:50 EST-0500_176_0.661.pth.tar',map_location=device)
# icnet = ICNet(nclass = 19).to(device)
# icnet.load_state_dict(model['model_state_dict'])
# icnet.eval()
# print('ICNet loaded')

## BiseNetV2
model = torch.load('/kaggle/input/bisenetv2_cityscapes_test/pytorch/default/1/model_final_v2_city.pth',map_location=device)
bisenetv2 = BiSeNetV2(19,aux_mode = 'eval').to(device)
bisenetv2.load_state_dict(model, strict=False)
bisenetv2.eval()
print('BisNetV2 loaded')

import torch
from omegaconf import OmegaConf
from hydra.utils import instantiate
from sam2.sam2_image_predictor import SAM2ImagePredictor

device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg      = OmegaConf.load("/kaggle/input/segment-anything-2/pytorch/sam2-hiera-tiny/1/sam2_hiera_t.yaml")
ckpt     = torch.load("/kaggle/input/segment-anything-2/pytorch/sam2-hiera-tiny/1/sam2_hiera_tiny.pt", map_location=device)

# 1) instantiate the bare model
sam2_model = instantiate(cfg.model).to(device)

# 2) load its weights
#   checkpoint dict usually has a "model" key
sam2_model.load_state_dict(ckpt.get("model", ckpt), strict=False)

# 3) wrap in predictor
predictor = SAM2ImagePredictor(sam2_model)
print("SAM2 tiny loaded via manual state_dict!")

## segformer from huggingface
import os, io, contextlib
from transformers.utils import logging as hf_logging
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation

# (A) kill progress bars & tokenizers chatter
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# (B) silence HF logging completely
hf_logging.set_verbosity_error()           # or set_verbosity(hf_logging.CRITICAL)
hf_logging.disable_default_handler()       # prevent adding console handlers
hf_logging.enable_propagation()            # keep logs from re-adding handlers

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# (C) suppress any stdout/stderr printed during load (belt & suspenders)
with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
    processor = SegformerImageProcessor.from_pretrained(
        "nvidia/segformer-b5-finetuned-cityscapes-1024-1024"
    )
    segformer = SegformerForSemanticSegmentation.from_pretrained(
        "nvidia/segformer-b5-finetuned-cityscapes-1024-1024"
    )

segformer.to(device).eval()
print("segformer loaded")

PIDNet-s Model loaded
PIDNet-m Model loaded
PIDNet-L loaded
BisNetV2 loaded
SAM2 tiny loaded via manual state_dict!
segformer loaded


In [23]:
import numpy as np
import torch

IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)

@torch.no_grad()
def hf_segformer_forward(segformer_model, hf_processor, x_bchw, already_normalized=False):
    """
    x_bchw: torch.FloatTensor [B,3,H,W]
    Returns: logits tensor [B, 19, H/4, W/4] for Cityscapes model.
    """
    x = x_bchw.detach().to(segformer_model.device)

    # If caller gave ImageNet-normalized tensors, revert to 0..1 so the HF processor can normalize once.
    if already_normalized:
        x = (x * IMAGENET_STD.to(x.device)) + IMAGENET_MEAN.to(x.device)
        x = x.clamp(0, 1)

    # Convert to uint8 images for processor
    imgs = (x.clamp(0,1) * 255.0).round().to(torch.uint8)          # [B,3,H,W]
    imgs = imgs.permute(0,2,3,1).cpu().numpy().astype(np.uint8)     # [B,H,W,3]

    inputs = hf_processor(images=list(imgs), return_tensors='pt', do_resize=False)
    pixel_values = inputs['pixel_values'].to(segformer_model.device)
    out = segformer_model(pixel_values=pixel_values)  # SemanticSegmenterOutput
    return out.logits  # tensor [B, num_labels, H/4, W/4]


### With adversarial patch

In [26]:
import numpy as np
import torch
import torch.nn.functional as F
from copy import deepcopy
from tqdm import tqdm
import gc
from tabulate import tabulate

mean_standard = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std_standard = np.array([0.229, 0.224, 0.225], dtype=np.float32)
x = (2048 - 200) // 2
y = (1024 - 200) // 2
x_end = x + 200
y_end = y + 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize metrics for all models
metric_pidnet_s = SegmentationMetric(config)
metric_pidnet_m = SegmentationMetric(config)
metric_pidnet_l = SegmentationMetric(config)
metric_bisenetv2 = SegmentationMetric(config)
metric_sam2 = SegmentationMetric(config)
metric_segformer = SegmentationMetric(config)

# Dictionary to store mIoU results
data = {}

# Iterate over patches
for pat in ['pidnet_s']:  # Modify as needed to include other patches
    patch = patches[pat]
    
    # Reset all metrics
    metric_pidnet_s.reset()
    metric_pidnet_m.reset()
    metric_pidnet_l.reset()
    metric_bisenetv2.reset()
    metric_sam2.reset()
    metric_segformer.reset()
    
    print(f'Computing for patch: {pat}')
    
    for iter, batches in tqdm(enumerate(val_dataloader)):
        image_standard, label, _, _, _ = batches
        label_patched = deepcopy(label)

        # Apply adversarial patch
        image_standard[:, :, y:y_end, x:x_end] = patch
        label_patched[:, y:y_end, x:x_end] = config.train.ignore_label
        
        # Move to device
        image_standard = image_standard.to(device)
        label_patched = label_patched.to(device)

        # PIDNet-S
        outputs = pidnet_s(image_standard)
        output = F.interpolate(
            outputs[config.test.output_index_pidnet], 
            size=label.shape[-2:],
            mode='bilinear', 
            align_corners=True
        )
        metric_pidnet_s.update(output, label_patched)

        # PIDNet-M
        outputs = pidnet_m(image_standard)
        output = F.interpolate(
            outputs[config.test.output_index_pidnet], 
            size=label.shape[-2:],
            mode='bilinear', 
            align_corners=True
        )
        metric_pidnet_m.update(output, label_patched)

        # PIDNet-L
        outputs = pidnet_l(image_standard)
        output = F.interpolate(
            outputs[config.test.output_index_pidnet], 
            size=label.shape[-2:],
            mode='bilinear', 
            align_corners=True
        )
        metric_pidnet_l.update(output, label_patched)

        # BiseNetV2
        outputs = bisenetv2(image_standard)
        output = outputs[config.test.output_index_bisenet]
        metric_bisenetv2.update(output, label_patched)

        # SAM2
        img_np = image_standard.squeeze(0).permute(1, 2, 0).cpu().numpy()
        img_np = (img_np * std_standard + mean_standard) * 255.0
        img_np = img_np.clip(0, 255).astype(np.uint8)
        
        predictor.set_image(img_np)
        H, W, _ = img_np.shape
        pts = np.array([[W // 2, H // 2]])
        labels = np.array([1])
        masks, _, _ = predictor.predict(
            point_coords=pts, 
            point_labels=labels, 
            multimask_output=False
        )
        pred_mask = torch.from_numpy(masks[0]).unsqueeze(0).unsqueeze(0).to(device).float()
        metric_sam2.update(pred_mask, label_patched)

        # SegFormer
        with torch.no_grad():
            logits = hf_segformer_forward(
                segformer_model = segformer,
                hf_processor    = processor,
                x_bchw          = image_standard,
                already_normalized = True   # set False if your images are raw 0..1
            )
            # Upsample to GT size and update your metric
            output = F.interpolate(
                logits, size=label.shape[-2:], mode='bilinear', align_corners=True
            )
            metric_segformer.update(output, label_patched)

        # Clean up
        del outputs, output, image_standard, label, batches
        gc.collect()
        torch.cuda.empty_cache()

    # Store mIoU results
    data[pat] = {
        'PIDNet-S': metric_pidnet_s.get()[1],
        'PIDNet-M': metric_pidnet_m.get()[1],
        'PIDNet-L': metric_pidnet_l.get()[1],
        'BiseNetV2': metric_bisenetv2.get()[1],
        'SAM2': metric_sam2.get()[1],
        'SegFormer': metric_segformer.get()[1]
    }

# Create and print formatted table
headers = ['Patch', 'PIDNet-S', 'PIDNet-M', 'PIDNet-L', 'BiseNetV2', 'SAM2', 'SegFormer']
table_data = []
for pat, metrics in data.items():
    row = [pat] + [f"{metrics[model]:.4f}" for model in ['PIDNet-S', 'PIDNet-M', 'PIDNet-L', 'BiseNetV2', 'SAM2', 'SegFormer']]
    table_data.append(row)

table = tabulate(table_data, headers=headers, tablefmt='pipe', floatfmt='.4f')
print("\nModel mIoU Results:")
print(table)

Computing for patch: pidnet_s


500it [24:07,  2.90s/it]


Model mIoU Results:
| Patch    |   PIDNet-S |   PIDNet-M |   PIDNet-L |   BiseNetV2 |   SAM2 |   SegFormer |
|:---------|-----------:|-----------:|-----------:|------------:|-------:|------------:|
| pidnet_s |     0.8170 |     0.8537 |     0.8931 |      0.6830 | 0.0196 |      0.8082 |


In [25]:
from tqdm import tqdm
from copy import deepcopy
mean_standard = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std_standard = np.array([0.229, 0.224, 0.225], dtype=np.float32)
x = (2048 - 200) // 2  # 924
y = (1024 - 200) // 2  # 412
x_end = x + 200
y_end = y + 200

# Initialize metric
metric_segformer = SegmentationMetric(config)  # Assuming config is defined

# Load SegFormer
segformer_b0 = load_segformer_local(
    variant_or_name="segformer_b0",
    ckpt_path=ckpt,  # Replace with your checkpoint path
    device=device,
    num_classes=config.dataset.num_classes,  # Likely 19 for Cityscapes
    decoder_channels=256,
    strict=True,
    verbose=True
)
segformer_b0.eval()

# Evaluation loop
data = {}
for pat in ['pidnet_s']:  # Single patch for testing
    patch = patches[pat]  # Assuming patches is a dict with patch tensors
    # Normalize patch if in [0, 255]
    if patch.max() > 1.0:
        patch = (patch / 255.0 - mean_standard) / std_standard
    metric_segformer.reset()
    print(f'Computing for: {pat}')
    for iter, batches in tqdm(enumerate(val_dataloader)):
        image_standard, label, _, _, _ = batches
        label_patched = deepcopy(label)

        # Add adversarial patch
        image_standard[:, :, y:y_end, x:x_end] = patch
        label_patched[:, y:y_end, x:x_end] = config.train.ignore_label

        # Move to device
        image_standard = image_standard.to(device)
        label_patched = label_patched.to(device)

        # SegFormer inference
        with torch.no_grad():
            outputs = segformer_logits(segformer_b0, image_standard)  # [B, 19, H/4, W/4]
            output = F.interpolate(
                outputs, size=label.shape[-2:], mode='bilinear', align_corners=True
            )
            metric_segformer.update(output, label_patched)

        del outputs, output, image_standard, label, batches
        gc.collect()
        torch.cuda.empty_cache()

    data[pat] = metric_segformer.get()[1]  # Get mIoU
    print(f'Results [SegFormer]: {data[pat]}')

AttributeError: 'dict' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

## Random patch for sssegformer 

In [15]:
mean_standard = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std_standard = np.array([0.229, 0.224, 0.225], dtype=np.float32)
x = (2048 - 200) // 2  # 924
y = (1024 - 200) // 2  # 412
x_end = x + 200
y_end = y + 200

# Initialize metric
metric_segformer = SegmentationMetric(config)  # Assuming config is defined

# Load SegFormer
segformer_b0 = load_segformer_local(
    variant_or_name="segformer_b0",
    ckpt_path=ckpt,  # Replace with your checkpoint path
    device=device,
    num_classes=config.dataset.num_classes,  # Likely 19 for Cityscapes
    decoder_channels=256,
    strict=True,
    verbose=True
)
segformer_b0.eval()

# Evaluation loop with random patch
metric_segformer.reset()
print('Computing for random patch')
for iter, batches in tqdm(enumerate(val_dataloader)):
    image_standard, label, _, _, _ = batches
    label_patched = deepcopy(label)

    # Generate random patch and normalize
    patch = torch.rand(3, 200, 200)  # Values in [0, 1]
    patch = (patch - mean_standard.reshape(3, 1, 1)) / std_standard.reshape(3, 1, 1) # Normalize to match image_standard
    image_standard[:, :, y:y_end, x:x_end] = patch
    label_patched[:, y:y_end, x:x_end] = config.train.ignore_label

    # Move to device
    image_standard = image_standard.to(device)
    label_patched = label_patched.to(device)

    # SegFormer inference
    with torch.no_grad():
        outputs = segformer_logits(segformer_b0, image_standard)  # [B, 19, H/4, W/4]
        output = F.interpolate(
            outputs, size=label.shape[-2:], mode='bilinear', align_corners=True
        )
        metric_segformer.update(output, label_patched)

    del outputs, output, image_standard, label, batches
    gc.collect()
    torch.cuda.empty_cache()

# Print result
print(f'SegFormer mIoU: {metric_segformer.get()[1]}')

[SegFormer] matched 191/191 keys | missing 0 | unexpected 0
Computing for random patch


500it [03:58,  2.10it/s]

SegFormer mIoU: 0.016241049394011497


### With random patch

In [ ]:
mean_standard = np.array([0.485, 0.456, 0.406],dtype = np.float32)
std_standard = np.array([0.229, 0.224, 0.225],dtype = np.float32)
x = (2048 - 200)//2
y = (1024 - 200)//2
x_end = x + 200
y_end = y + 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

metric_pidnet_m = SegmentationMetric(config)
metric_pidnet_s = SegmentationMetric(config)
# metric_icnet = SegmentationMetric(config)
metric_bisenetv2 = SegmentationMetric(config)
metric_sam2 = SegmentationMetric(config)

metric_pidnet_m.reset()
metric_pidnet_s.reset()
# metric_icnet.reset()
metric_bisenetv2.reset()
metric_sam2.reset()

for iter,batches in tqdm(enumerate(val_dataloader)):
  image_standard,label,_,_,_ = batches
  label_patched = deepcopy(label)

  ## random patch
  patch = torch.rand(3,200,200)
  image_standard[:,:, y:y_end, x:x_end] = patch
  label_patched[:, y:y_end, x:x_end] = config.train.ignore_label
  ## On device
  image_standard = image_standard.to(device)
  label_patched = label_patched.to(device)

  ##PIDNet-m
  outputs = pidnet_m(image_standard)
  size = label.shape
  output = F.interpolate(
                outputs[config.test.output_index_pidnet], size[-2:],
                mode='bilinear', align_corners=True
                        )
  metric_pidnet_m.update(output, label_patched)

  ## PIDNet-s
  outputs = pidnet_s(image_standard)
  size = label.shape
  output = F.interpolate(
                outputs[config.test.output_index_pidnet], size[-2:],
                mode='bilinear', align_corners=True
                        )
  metric_pidnet_s.update(output, label_patched)

  # ##ICNet
  # outputs = icnet(image_standard)
  # output = outputs[config.test.output_index_icnet]
  # metric_icnet.update(output,label_patched)

  ##BiseNetV2
  outputs = bisenetv2(image_standard)
  output = outputs[config.test.output_index_bisenet]
  metric_bisenetv2.update(output,label_patched)

    # run SAM 2 prediction
  predictor.set_image(img_np)
    # use a center-point prompt
  H, W, _ = img_np.shape
  pts = np.array([[W//2, H//2]])
  labels = np.array([1])
  masks, _, _ = predictor.predict(point_coords=pts, point_labels=labels, multimask_output=False)
    
    # convert to torch tensor and update metric
  pred_mask = torch.from_numpy(masks[0])       # H×W
  pred_mask = torch.from_numpy(masks[0]).unsqueeze(0).unsqueeze(0).to(device).float()
  metric_sam2.update(pred_mask, label_patched)

  del outputs,output,image_standard,label,batches
  gc.collect()
  torch.cuda.empty_cache()

print(f'PIDNet-M MIOU: {metric_pidnet_m.get()[1]}')
print(f'PIDNet-S MIOU: {metric_pidnet_s.get()[1]}')
print(f'BisNetV2 MIOU: {metric_bisenetv2.get()[1]}')

In [ ]:
print(f'SAM2 MIOU: {metric_sam2.get()[1]}')

### Without Patch

In [ ]:
mean_standard = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std_standard  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
device        = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Reset metrics
metric_pidnet_m   = SegmentationMetric(config)
metric_pidnet_s   = SegmentationMetric(config)
metric_bisenetv2  = SegmentationMetric(config)
metric_sam2       = SegmentationMetric(config)

# Single key for loop consistency (you can also just drop the outer loop entirely)
for pat in ['no_patch']:
    metric_pidnet_m.reset()
    metric_pidnet_s.reset()
    metric_bisenetv2.reset()
    metric_sam2.reset()
    print(f'Computing on unmodified images:')

    for batches in tqdm(val_dataloader, total=len(val_dataloader)):
        image, label, *_ = batches

        # send to device
        image = image.to(device)
        label = label.to(device)

        # --- PIDNet-m ---
        out_m = pidnet_m(image)
        out_m = F.interpolate(
            out_m[config.test.output_index_pidnet],
            size=label.shape[-2:], mode='bilinear', align_corners=True
        )
        metric_pidnet_m.update(out_m, label)

        # --- PIDNet-s ---
        out_s = pidnet_s(image)
        out_s = F.interpolate(
            out_s[config.test.output_index_pidnet],
            size=label.shape[-2:], mode='bilinear', align_corners=True
        )
        metric_pidnet_s.update(out_s, label)

        # --- BiseNetV2 ---
        out_b = bisenetv2(image)[config.test.output_index_bisenet]
        metric_bisenetv2.update(out_b, label)

        # --- SAM-2 evaluation on the full image ---
        # convert to H×W×3 uint8 for SAM
        img_np = image.squeeze(0).permute(1,2,0).cpu().numpy()
        img_np = (img_np * std_standard + mean_standard) * 255.0
        img_np = img_np.clip(0,255).astype(np.uint8)

        predictor.set_image(img_np)
        H, W, _ = img_np.shape
        pts      = np.array([[W//2, H//2]])
        lbls     = np.array([1])
        masks, _, _ = predictor.predict(
            point_coords=pts,
            point_labels=lbls,
            multimask_output=False
        )
        # [1,1,H,W]
        pred_mask = (
            torch.from_numpy(masks[0])
                 .unsqueeze(0).unsqueeze(0)
                 .to(device).float()
        )
        metric_sam2.update(pred_mask, label)

        # cleanup
        del out_m, out_s, out_b, image, label, masks, img_np
        torch.cuda.empty_cache()

    # report
    data = [
        metric_pidnet_m.get()[1],
        metric_pidnet_s.get()[1],
        metric_bisenetv2.get()[1],
        metric_sam2.get()[1]
    ]
    print(f'Results [PIDNet-M, PIDNet-S, BiseNetV2, SAM-2]: {data}')

## New

In [ ]:
!git clone https://github.com/SargamG/adversarial-patch-transferability

In [ ]:
!cp /kaggle/input/pidnet-s-weights/PIDNet_S_Cityscapes_test.pt /kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet
!cp /kaggle/input/pidnet-m-weights/PIDNet_M_Cityscapes_test.pt /kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet
!cp /kaggle/input/bisenetv2-weights/bisenetv2.pth /kaggle/working/adversarial-patch-transferability/pretrained_models/BisNetV2

In [ ]:
%cd /kaggle/working/adversarial-patch-transferability

In [ ]:
from pretrained_models.ICNet.icnet import ICNet
#config = get_config_from_yaml('/kaggle/working/adversarial-patch-transferability/configs/config.yaml')
icnet = ICNet(nclass = 19).to(device)
for name, _ in icnet.named_modules():
    print(name)

In [ ]:
for name, _ in bisenetv2.named_modules():
    print(name)

In [ ]:
for name, _ in pidnet_m.named_modules():
    print(name)

In [ ]:
for name, _ in pidnet_s.named_modules():
    print(name)

In [ ]:
cityscape_val = Cityscapes(
          root = config.dataset.root,
          list_path = config.dataset.val,
          num_classes = config.dataset.num_classes,
          multi_scale = False,
          flip = False,
          ignore_label = config.train.ignore_label,
          base_size = config.train.base_size,
          crop_size = (config.train.height,config.train.width),
        )

val_dataloader = torch.utils.data.DataLoader(dataset=cityscape_val,
                                            batch_size=1,
                                            shuffle=True,
                                            num_workers=config.train.num_workers,
                                            pin_memory=config.train.pin_memory,
                                            drop_last=config.train.drop_last)

In [ ]:
cityscape_train = Cityscapes(
          root = config.dataset.root,
          list_path = config.dataset.train,
          num_classes = config.dataset.num_classes,
          multi_scale = config.train.multi_scale,
          flip = config.train.flip,
          ignore_label = config.train.ignore_label,
          base_size = config.train.base_size,
          crop_size = (config.train.height,config.train.width),
          scale_factor = config.train.scale_factor
        )
train_dataloader = torch.utils.data.DataLoader(dataset=cityscape_train,
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=config.train.num_workers,
                                              pin_memory=config.train.pin_memory,
                                              drop_last=config.train.drop_last)

In [ ]:
def hook(module, input, output):
    global feature_maps
    feature_maps = output
    feature_maps.retain_grad()
layer_name = 'pretrained.layer2.3.relu'
for name, module in icnet.named_modules():
    if name == layer_name:
        module.register_forward_hook(hook)
        break

In [ ]:
for iter,batches in tqdm(enumerate(train_dataloader)):
    image_standard,label,_,_,_, idx = batches

    ## On device
    image_standard = image_standard.to(device)
    label = label.to(device)

    ##PIDNet-m
    outputs = icnet(image_standard)
    break
feature_maps.shape

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('/kaggle/working/adversarial-patch-transferability')
from utils.utils import setup_logging, get_config_from_yaml, process_config, print_config
from trainer.trainer import PatchTrainer
import torch
import pickle

In [ ]:
model = 'pidnet_s'
## getting the config file
config = get_config_from_yaml('configs/config.yaml',model = model)

## processing config to initialize directories for logs etc
main_logger = process_config(config)

## training
train_obj = PatchTrainer(config,main_logger,model)
train_obj.register_forward_hook()
H = train_obj.get_agg_gradient()
print(H)

In [ ]:
path='/kaggle/working/logs/H.pt'
torch.save(H,path)

In [ ]:
import torch
import gzip
path = '/kaggle/working/logs/H_tensor.pt.gz'
with gzip.open(path, 'wb') as f:
    torch.save(H, f)


In [ ]:
import yaml

yaml_path = "/kaggle/working/adversarial-patch-transferability/configs/config.yaml"

# Read
with open(yaml_path, 'r') as file:
    conf = yaml.safe_load(file)

print(conf)  # View content


In [ ]:
conf["train"]["batch_size"]=1
with open(yaml_path, 'w') as file:
    yaml.safe_dump(conf, file)


In [ ]:
model = 'pidnet_s'
## getting the config file
config = get_config_from_yaml('configs/config.yaml',model = model)

## processing config to initialize directories for logs etc
main_logger = process_config(config)

## training
train_obj = PatchTrainer(config,main_logger,model)
train_obj.register_forward_hook()
H

In [ ]:
save = train_obj.train(H)
pickle.dump( save, open(config.experiment.log_patch_address+config.model.name+".p", "wb" ) )

In [ ]:
pickle.dump( save, open(config.experiment.log_patch_address+config.model.name+"_bbfa"+".p", "wb" ) )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import gc
sys.path.append('/kaggle/working/adversarial-patch-transferability')
from utils.utils import setup_logging, get_config_from_yaml, process_config, print_config
from dataset.cityscapes import Cityscapes
from metrics.performance import SegmentationMetric
from utils.helper import val_plot
from patch.create import Patch
#from metrics.performance import SegmentationMetric


from pretrained_models.ICNet.icnet import ICNet
from pretrained_models.BisNetV2.model import BiSeNetV2
from pretrained_models.PIDNet.model import PIDNet, get_pred_model
import pickle
from copy import deepcopy
# from trainer.trainer import Trainer
# import torch
from tqdm import tqdm
config = get_config_from_yaml('/kaggle/working/adversarial-patch-transferability/configs/config.yaml')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
pidnet_s_p = pickle.load(open( "/kaggle/working/adversarial-patch-transferability/Experiments/pidnet_s_bbfa.p", "rb" ))[0]
patches = {
    'pidnet_s':pidnet_s_p
}

for patch in patches:
  patches[patch] = patches[patch].to(device)
  print(patch)

In [ ]:
cityscape_val = Cityscapes(
          root = config.dataset.root,
          list_path = config.dataset.val,
          num_classes = config.dataset.num_classes,
          multi_scale = False,
          flip = False,
          ignore_label = config.train.ignore_label,
          base_size = config.train.base_size,
          crop_size = (config.train.height,config.train.width),
        )

val_dataloader = torch.utils.data.DataLoader(dataset=cityscape_val,
                                            batch_size=1,
                                            shuffle=True,
                                            num_workers=config.train.num_workers,
                                            pin_memory=config.train.pin_memory,
                                            drop_last=config.train.drop_last)

In [ ]:
## PIDNet-s
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet/PIDNet_S_Cityscapes_test.pt',map_location=device)
pidnet_s = get_pred_model(name = 'pidnet_s', num_classes = 19).to(device)
if 'state_dict' in model:
    model = model['state_dict']
model_dict = pidnet_s.state_dict()
model = {k[6:]: v for k, v in model.items() # k[6:] to start after model. in key names
                    if k[6:] in model_dict.keys()}

pidnet_s.load_state_dict(model)
pidnet_s.eval()
print('PIDNet Model loaded')

## PIDNet-m
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet/PIDNet_M_Cityscapes_test.pt',map_location=device)
pidnet_m = get_pred_model(name = 'pidnet_m', num_classes = 19).to(device)
if 'state_dict' in model:
    model = model['state_dict']
model_dict = pidnet_m.state_dict()
model = {k[6:]: v for k, v in model.items() # k[6:] to start after model. in key names
                    if k[6:] in model_dict.keys()}

pidnet_m.load_state_dict(model)
pidnet_m.eval()
print('PIDNet Model loaded')

# ## ICNet
# model = torch.load('/content/drive/MyDrive/Colab Notebooks/1_Papers/3_Attack_generation/pretrained_models/ICNet/Copy of resnet50_2024-12-22 08:52:50 EST-0500_176_0.661.pth.tar',map_location=device)
# icnet = ICNet(nclass = 19).to(device)
# icnet.load_state_dict(model['model_state_dict'])
# icnet.eval()
# print('ICNet loaded')

## BiseNetV2
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/BisNetV2/bisenetv2.pth',map_location=device)
bisenetv2 = BiSeNetV2(19,aux_mode = 'eval').to(device)
bisenetv2.load_state_dict(model, strict=False)
bisenetv2.eval()
print('BisNetV2 loaded')

In [ ]:
mean_standard = np.array([0.485, 0.456, 0.406],dtype = np.float32)
std_standard = np.array([0.229, 0.224, 0.225],dtype = np.float32)
x = (2048 - 200)//2
y = (1024 - 200)//2
x_end = x + 200
y_end = y + 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
metric_pidnet_m = SegmentationMetric(config)
metric_pidnet_s = SegmentationMetric(config)
# metric_icnet = SegmentationMetric(config)
metric_bisenetv2 = SegmentationMetric(config)


data = {}
for pat in ['pidnet_s']:
  patch = patches[pat]
  metric_pidnet_m.reset()
  metric_pidnet_s.reset()
  # metric_icnet.reset()
  metric_bisenetv2.reset()
  print(f'Computing for: {pat}')
  temp = []
  for iter,batches in tqdm(enumerate(val_dataloader)):
    image_standard,label,_,_,_,_ = batches
    label_patched = deepcopy(label)

    ## adding patch
    image_standard[:,:, y:y_end, x:x_end] = patch
    label_patched[:, y:y_end, x:x_end] = config.train.ignore_label
    ## On device
    image_standard = image_standard.to(device)
    label_patched = label_patched.to(device)

    ##PIDNet-m
    outputs = pidnet_m(image_standard)
    size = label.shape
    output = F.interpolate(
                  outputs[config.test.output_index_pidnet], size[-2:],
                  mode='bilinear', align_corners=True
                          )
    metric_pidnet_m.update(output, label_patched)

    ## PIDNet-s
    outputs = pidnet_s(image_standard)
    size = label.shape
    output = F.interpolate(
                  outputs[config.test.output_index_pidnet], size[-2:],
                  mode='bilinear', align_corners=True
                          )
    metric_pidnet_s.update(output, label_patched)

    # ##ICNet
    # outputs = icnet(image_standard)
    # output = outputs[config.test.output_index_icnet]
    # metric_icnet.update(output,label_patched)

    ##BiseNetV2
    outputs = bisenetv2(image_standard)
    output = outputs[config.test.output_index_bisenet]
    metric_bisenetv2.update(output,label_patched)

    del outputs,output,image_standard,label,batches
    gc.collect()
    torch.cuda.empty_cache()

  data[pat] = [
      metric_pidnet_m.get()[1],
      metric_pidnet_s.get()[1],
      # metric_icnet.get()[1],
      metric_bisenetv2.get()[1],
  ]
  print(data[pat])

### Visualising feature maps after trained patch

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import gc
sys.path.append('/kaggle/working/adversarial-patch-transferability')
from utils.utils import setup_logging, get_config_from_yaml, process_config, print_config
from dataset.cityscapes import Cityscapes
from metrics.performance import SegmentationMetric
from utils.helper import val_plot
from patch.create import Patch
#from metrics.performance import SegmentationMetric


from pretrained_models.ICNet.icnet import ICNet
from pretrained_models.BisNetV2.model import BiSeNetV2
from pretrained_models.PIDNet.model import PIDNet, get_pred_model
import pickle
from copy import deepcopy
# from trainer.trainer import Trainer
# import torch
from tqdm import tqdm
config = get_config_from_yaml('/kaggle/working/adversarial-patch-transferability/configs/config.yaml')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
cityscape_val = Cityscapes(
          root = config.dataset.root,
          list_path = config.dataset.val,
          num_classes = config.dataset.num_classes,
          multi_scale = False,
          flip = False,
          ignore_label = config.train.ignore_label,
          base_size = config.train.base_size,
          crop_size = (config.train.height,config.train.width),
        )

val_dataloader = torch.utils.data.DataLoader(dataset=cityscape_val,
                                            batch_size=1,
                                            shuffle=False,
                                            num_workers=config.train.num_workers,
                                            pin_memory=config.train.pin_memory,
                                            drop_last=config.train.drop_last)

In [ ]:
## PIDNet-s
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet/PIDNet_S_Cityscapes_test.pt',map_location=device)
pidnet_s = get_pred_model(name = 'pidnet_s', num_classes = 19).to(device)
if 'state_dict' in model:
    model = model['state_dict']
model_dict = pidnet_s.state_dict()
model = {k[6:]: v for k, v in model.items() # k[6:] to start after model. in key names
                    if k[6:] in model_dict.keys()}

pidnet_s.load_state_dict(model)
pidnet_s.eval()
print('PIDNet Model loaded')

## PIDNet-m
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet/PIDNet_M_Cityscapes_test.pt',map_location=device)
pidnet_m = get_pred_model(name = 'pidnet_m', num_classes = 19).to(device)
if 'state_dict' in model:
    model = model['state_dict']
model_dict = pidnet_m.state_dict()
model = {k[6:]: v for k, v in model.items() # k[6:] to start after model. in key names
                    if k[6:] in model_dict.keys()}

pidnet_m.load_state_dict(model)
pidnet_m.eval()
print('PIDNet Model loaded')

# ## ICNet
# model = torch.load('/content/drive/MyDrive/Colab Notebooks/1_Papers/3_Attack_generation/pretrained_models/ICNet/Copy of resnet50_2024-12-22 08:52:50 EST-0500_176_0.661.pth.tar',map_location=device)
# icnet = ICNet(nclass = 19).to(device)
# icnet.load_state_dict(model['model_state_dict'])
# icnet.eval()
# print('ICNet loaded')

## BiseNetV2
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/BisNetV2/bisenetv2.pth',map_location=device)
bisenetv2 = BiSeNetV2(19,aux_mode = 'eval').to(device)
bisenetv2.load_state_dict(model, strict=False)
bisenetv2.eval()
print('BisNetV2 loaded')

In [ ]:
feature_maps_attack = []
feature_maps = []

In [ ]:
print(len(feature_maps_attack))

In [ ]:
def hook_attack(module, input, output):
    feature_maps_attack.append(output.detach().cpu()) 

def hook(module, input, output):
    feature_maps.append(output.detach().cpu())
    
layer_name = 'layer3.2.bn2'
for name, module in pidnet_m.named_modules():
    if name == layer_name:
        module.register_forward_hook(hook_attack)
        break

In [ ]:
pidnet_s_p = pickle.load(open( "/kaggle/working/adversarial-patch-transferability/Experiments/pidnet_s_bbfa.p", "rb" ))[0]
patches = {
    'pidnet_s':pidnet_s_p
}

for patch in patches:
  patches[patch] = patches[patch].to(device)
  print(patch)

In [ ]:
mean_standard = np.array([0.485, 0.456, 0.406],dtype = np.float32)
std_standard = np.array([0.229, 0.224, 0.225],dtype = np.float32)
x = (2048 - 200)//2
y = (1024 - 200)//2
x_end = x + 200
y_end = y + 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for pat in ['pidnet_s']:
  patch = patches[pat]
  print(f'Computing for: {pat}')
  for iter,batches in tqdm(enumerate(val_dataloader)):
    image_standard,label,_,_,_,idx = batches
    label_patched = deepcopy(label)

    ## adding patch
    image_standard[:,:, y:y_end, x:x_end] = patch
    label_patched[:, y:y_end, x:x_end] = config.train.ignore_label
    ## On device
    image_standard = image_standard.to(device)
    label_patched = label_patched.to(device)

    ##PIDNet-m
    outputs = pidnet_m(image_standard)
    size = label.shape
    output = F.interpolate(
                  outputs[config.test.output_index_pidnet], size[-2:],
                  mode='bilinear', align_corners=True
                          )
    break

In [ ]:
mean_standard = np.array([0.485, 0.456, 0.406],dtype = np.float32)
std_standard = np.array([0.229, 0.224, 0.225],dtype = np.float32)
x = (2048 - 200)//2
y = (1024 - 200)//2
x_end = x + 200
y_end = y + 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for iter,batches in tqdm(enumerate(val_dataloader)):
  image_standard,label,_,_,_,_ = batches
  label_patched = deepcopy(label)

  ## random patch
  patch = torch.rand(3,200,200)
  image_standard[:,:, y:y_end, x:x_end] = patch
  label_patched[:, y:y_end, x:x_end] = config.train.ignore_label
  ## On device
  image_standard = image_standard.to(device)
  label_patched = label_patched.to(device)

  ##PIDNet-s
  outputs = pidnet_s(image_standard)
  size = label.shape
  output = F.interpolate(
                outputs[config.test.output_index_pidnet], size[-2:],
                mode='bilinear', align_corners=True
                        )

In [ ]:
print(len(feature_maps_attack))
print(len(feature_maps))

In [ ]:
## for layer 3
import matplotlib.pyplot as plt

# Get the first feature map batch
fm = feature_maps_attack[0][0]  # shape: [C, H, W], get first image in batch

# Show first 8 channels
plt.figure(figsize=(16, 8))
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(fm[i], cmap='viridis')
    plt.axis('off')
    plt.title(f'Channel {i}')
plt.suptitle("Feature Maps from Layer after Adversarial Patch")
plt.tight_layout()
plt.show()
## On pidnet-s

In [ ]:
## for layer 3
import matplotlib.pyplot as plt

# Get the first feature map batch
fm = feature_maps[0][0]  # shape: [C, H, W], get first image in batch

# Show first 8 channels
plt.figure(figsize=(16, 8))
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(fm[i], cmap='viridis')
    plt.axis('off')
    plt.title(f'Channel {i}')
plt.suptitle("Feature Maps from Layer after Random Patch on PIDNet-S")
plt.tight_layout()
plt.show()

In [ ]:
## For layer 1
import matplotlib.pyplot as plt

# Get the first feature map batch
fm = feature_maps_attack[0][0]  # shape: [C, H, W], get first image in batch

# Show first 8 channels
plt.figure(figsize=(16, 8))
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(fm[i], cmap='viridis')
    plt.axis('off')
    plt.title(f'Channel {i}')
plt.suptitle("Feature Maps from Layer after Adversarial Patch")
plt.tight_layout()
plt.show()
## On pidnet-s

In [ ]:
## For layer 1
import matplotlib.pyplot as plt

# Get the first feature map batch
fm = feature_maps[0][0]  # shape: [C, H, W], get first image in batch

# Show first 8 channels
plt.figure(figsize=(16, 8))
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(fm[i], cmap='viridis')
    plt.axis('off')
    plt.title(f'Channel {i}')
plt.suptitle("Feature Maps from Layer after Random Patch on PIDNet-S")
plt.tight_layout()
plt.show()

In [ ]:
## For layer 1
import matplotlib.pyplot as plt

# Get the first feature map batch
fm = feature_maps_attack[0][0]  # shape: [C, H, W], get first image in batch

# Show first 8 channels
plt.figure(figsize=(16, 8))
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(fm[i], cmap='viridis')
    plt.axis('off')
    plt.title(f'Channel {i}')
plt.suptitle("Feature Maps from Layer after Adversarial Patch")
plt.tight_layout()
plt.show()
## On pidnet-m

In [ ]:
## For layer 3
import matplotlib.pyplot as plt

# Get the first feature map batch
fm = feature_maps_attack[0][0]  # shape: [C, H, W], get first image in batch

# Show first 8 channels
plt.figure(figsize=(16, 8))
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(fm[i], cmap='viridis')
    plt.axis('off')
    plt.title(f'Channel {i}')
plt.suptitle("Feature Maps from Layer after Adversarial Patch")
plt.tight_layout()
plt.show()
## On pidnet-m

In [ ]:
feature_maps = []

def hook_fn(module, input, output):
    feature_maps.append(output.detach().cpu())  # Save to CPU to avoid memory issues


In [ ]:
layer_name = 'layer3.2.bn2'  # replace as needed

# Find and hook the layer
for name, module in model.model.named_modules():  # assuming your model is `model.model`
    if name == layer_name:
        module.register_forward_hook(hook_fn)
        break


In [ ]:
save[1].shape

In [ ]:
# If file is in the current directory
path='/kaggle/working/adversarial-patch-transferability/trainer/trainer.py'
with open(path, 'r') as file:
    contents = file.read()

print(contents)


## Comparing feature maps

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import gc
sys.path.append('/kaggle/working/adversarial-patch-transferability')
from utils.utils import setup_logging, get_config_from_yaml, process_config, print_config
from dataset.cityscapes import Cityscapes
from metrics.performance import SegmentationMetric
from utils.helper import val_plot
from patch.create import Patch
#from metrics.performance import SegmentationMetric


from pretrained_models.ICNet.icnet import ICNet
from pretrained_models.BisNetV2.model import BiSeNetV2
from pretrained_models.PIDNet.model import PIDNet, get_pred_model
import pickle
from copy import deepcopy
# from trainer.trainer import Trainer
# import torch
from tqdm import tqdm
config = get_config_from_yaml('/kaggle/working/adversarial-patch-transferability/configs/config.yaml')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
cityscape_val = Cityscapes(
          root = config.dataset.root,
          list_path = config.dataset.val,
          num_classes = config.dataset.num_classes,
          multi_scale = False,
          flip = False,
          ignore_label = config.train.ignore_label,
          base_size = config.train.base_size,
          crop_size = (config.train.height,config.train.width),
        )

val_dataloader = torch.utils.data.DataLoader(dataset=cityscape_val,
                                            batch_size=1,
                                            shuffle=False,
                                            num_workers=config.train.num_workers,
                                            pin_memory=config.train.pin_memory,
                                            drop_last=config.train.drop_last)

In [ ]:
## PIDNet-s
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet/PIDNet_S_Cityscapes_test.pt',map_location=device)
pidnet_s = get_pred_model(name = 'pidnet_s', num_classes = 19).to(device)
if 'state_dict' in model:
    model = model['state_dict']
model_dict = pidnet_s.state_dict()
model = {k[6:]: v for k, v in model.items() # k[6:] to start after model. in key names
                    if k[6:] in model_dict.keys()}

pidnet_s.load_state_dict(model)
pidnet_s.eval()
print('PIDNet Model loaded')

## PIDNet-m
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet/PIDNet_M_Cityscapes_test.pt',map_location=device)
pidnet_m = get_pred_model(name = 'pidnet_m', num_classes = 19).to(device)
if 'state_dict' in model:
    model = model['state_dict']
model_dict = pidnet_m.state_dict()
model = {k[6:]: v for k, v in model.items() # k[6:] to start after model. in key names
                    if k[6:] in model_dict.keys()}

pidnet_m.load_state_dict(model)
pidnet_m.eval()
print('PIDNet Model loaded')

# ## ICNet
# model = torch.load('/content/drive/MyDrive/Colab Notebooks/1_Papers/3_Attack_generation/pretrained_models/ICNet/Copy of resnet50_2024-12-22 08:52:50 EST-0500_176_0.661.pth.tar',map_location=device)
# icnet = ICNet(nclass = 19).to(device)
# icnet.load_state_dict(model['model_state_dict'])
# icnet.eval()
# print('ICNet loaded')

## BiseNetV2
model = torch.load('/kaggle/working/adversarial-patch-transferability/pretrained_models/BisNetV2/bisenetv2.pth',map_location=device)
bisenetv2 = BiSeNetV2(19,aux_mode = 'eval').to(device)
bisenetv2.load_state_dict(model, strict=False)
bisenetv2.eval()
print('BisNetV2 loaded')

In [ ]:
pidnet_s_p = pickle.load(open( "/kaggle/working/adversarial-patch-transferability/Experiments/pidnet_s_bbfa.p", "rb" ))[0]
patches = {
    'pidnet_s':pidnet_s_p
}

for patch in patches:
  patches[patch] = patches[patch].to(device)
  print(patch)

In [ ]:
feature_maps_pidnet_s =torch.randn(64,32,64)
feature_maps_pidnet_m = torch.randn(128,32,64)
feature_maps_bisenetv2 = torch.randn(128,32,64)

def hook_pidnet_s(module, input, output):
    feature_maps_pidnet_s = output

def hook_pidnet_m(module, input, output):
    feature_maps_pidnet_m = output

def hook_bisenetv2(module, input, output):
    feature_maps_bisenetv2 = output


In [ ]:
target_layer1 = 'compression4.1'  # for PIDNet-S, e.g.
target_layer2 = 'compression4.1'  # for BiSeNet
target_layer3 = 'segment.S5_5.conv_last.relu'  # for ResNet-like

for name, module in pidnet_s.named_modules():
    if name == target_layer1:
        module.register_forward_hook(hook_pidnet_s)

for name, module in pidnet_m.named_modules():
    if name == target_layer2:
        module.register_forward_hook(hook_pidnet_m)

for name, module in bisenetv2.named_modules():
    if name == target_layer3:
        module.register_forward_hook(hook_bisenetv2)


In [ ]:
mean_standard = np.array([0.485, 0.456, 0.406],dtype = np.float32)
std_standard = np.array([0.229, 0.224, 0.225],dtype = np.float32)
x = (2048 - 200)//2
y = (1024 - 200)//2
x_end = x + 200
y_end = y + 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = {}
temp = []
for iter,batches in tqdm(enumerate(val_dataloader)):
    image_standard,label,_,_,_,_ = batches
    
    ## On device
    image_standard = image_standard.to(device)
    label = label.to(device)
    
    ##PIDNet-m
    outputs = pidnet_m(image_standard)
    size = label.shape
    output = F.interpolate(
                  outputs[config.test.output_index_pidnet], size[-2:],
                  mode='bilinear', align_corners=True
                          )
    
    ## PIDNet-s
    outputs = pidnet_s(image_standard)
    size = label.shape
    output = F.interpolate(
                  outputs[config.test.output_index_pidnet], size[-2:],
                  mode='bilinear', align_corners=True
                          )
    
    # ##ICNet
    # outputs = icnet(image_standard)
    # output = outputs[config.test.output_index_icnet]
    # metric_icnet.update(output,label_patched)
    
    ##BiseNetV2
    outputs = bisenetv2(image_standard)
    output = outputs[config.test.output_index_bisenet]
    
    del outputs,output,image_standard,label,batches
    gc.collect()
    torch.cuda.empty_cache()
    
    break

In [ ]:
print(feature_maps_pidnet_s.shape)
print(feature_maps_pidnet_m.shape)
print(feature_maps_bisenetv2.shape)

In [ ]:
import torch.nn.functional as F

# Resize function (to common shape: C=64, H=64, W=64)
def resize_feature_map(fmap, target_channels=64, target_size=(64, 128)):
    fmap = fmap.unsqueeze(0)  # (1, C, H, W)
    
    # First: spatial resize
    fmap = F.interpolate(fmap, size=target_size, mode='bilinear', align_corners=False)

    # Then: channel resize using 1x1 conv if needed
    if fmap.shape[1] != target_channels:
        conv = torch.nn.Conv2d(fmap.shape[1], target_channels, kernel_size=1)
        fmap = conv(fmap)
    
    return fmap  # (1,C, H, W)

# Resize all to (64, 64, 64)
pidnet_s_resized = resize_feature_map(feature_maps_pidnet_s, 128, (32,64))
pidnet_m_resized = resize_feature_map(feature_maps_pidnet_m, 128, (32,64))
bisenetv2_resized = resize_feature_map(feature_maps_bisenetv2, 128, (32,64))
print(pidnet_s_resized.shape)
# Flatten to vectors
fm1 = pidnet_s_resized.flatten()
fm2 = pidnet_m_resized.flatten()
fm3 = bisenetv2_resized.flatten()

In [ ]:
fm1.shape

In [ ]:
# Compute cosine similarity between corresponding channels
def mean_cosine_similarity(fmA, fmB):
    sim = F.cosine_similarity(fmA, fmB, dim=0)  # channel-wise
    return sim.mean().item()

In [ ]:
print("PIDNet-S vs PIDNet-M:", mean_cosine_similarity(fm1, fm2))
print("PIDNet-S vs BiSeNetv2:", mean_cosine_similarity(fm1, fm3))
print("PIDNet-M vs BiSeNetv2:", mean_cosine_similarity(fm2, fm3))

### Comparing feature maps using random and adversarial patch on PIDNet-S

In [ ]:
feature_maps_pidnet_s = []
feature_maps_pidnet_s_attack = []

In [ ]:
def hook_pidnet_s(module, input, output):
    feature_maps_pidnet_s.append(output.detach().cpu())

In [ ]:
target_layer1 = 'layer3.2.bn2'  # for PIDNet-S

for name, module in pidnet_s.named_modules():
    if name == target_layer1:
        module.register_forward_hook(hook_pidnet_s)

In [ ]:
mean_standard = np.array([0.485, 0.456, 0.406],dtype = np.float32)
std_standard = np.array([0.229, 0.224, 0.225],dtype = np.float32)
x = (2048 - 200)//2
y = (1024 - 200)//2
x_end = x + 200
y_end = y + 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for pat in ['pidnet_s']:
  patch = patches[pat]
  print(f'Computing for: {pat}')
  for iter,batches in tqdm(enumerate(val_dataloader)):
    image_standard,label,_,_,_,idx = batches
    label_patched = deepcopy(label)

    ## adding patch
    image_standard[:,:, y:y_end, x:x_end] = patch
    label_patched[:, y:y_end, x:x_end] = config.train.ignore_label
    ## On device
    image_standard = image_standard.to(device)
    label_patched = label_patched.to(device)

    ##PIDNet-s
    outputs = pidnet_s(image_standard)
    size = label.shape
    output = F.interpolate(
                  outputs[config.test.output_index_pidnet], size[-2:],
                  mode='bilinear', align_corners=True
                          )
    break

In [ ]:
mean_standard = np.array([0.485, 0.456, 0.406],dtype = np.float32)
std_standard = np.array([0.229, 0.224, 0.225],dtype = np.float32)
x = (2048 - 200)//2
y = (1024 - 200)//2
x_end = x + 200
y_end = y + 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for pat in ['pidnet_s']:
  patch = patches[pat]
  print(f'Computing for: {pat}')
  for iter,batches in tqdm(enumerate(val_dataloader)):
    image_standard,label,_,_,_,idx = batches
    label_patched = deepcopy(label)

    ## adding patch
    patch = torch.rand(3,200,200)
    image_standard[:,:, y:y_end, x:x_end] = patch
    label_patched[:, y:y_end, x:x_end] = config.train.ignore_label
    ## On device
    image_standard = image_standard.to(device)
    label_patched = label_patched.to(device)

    ##PIDNet-s
    outputs = pidnet_s(image_standard)
    size = label.shape
    output = F.interpolate(
                  outputs[config.test.output_index_pidnet], size[-2:],
                  mode='bilinear', align_corners=True
                          )
    break

In [ ]:
print(len(feature_maps_pidnet_s_attack))
print(len(feature_maps_pidnet_s_attack))

In [ ]:
import torch.nn.functional as F

def flatten_and_normalize(fm):
    # shape: [C, H, W] → [C, H*W]
    fm_flat = fm.view(fm.shape[0], -1)
    return F.normalize(fm_flat, dim=1)

# Get feature map of the first image in batch
fm1 = flatten_and_normalize(feature_maps_pidnet_s[0][0])
fm2 = flatten_and_normalize(feature_maps_pidnet_s_attack[0][0])

# Compute cosine similarity between corresponding channels
def mean_cosine_similarity(fmA, fmB):
    sim = F.cosine_similarity(fmA, fmB, dim=1)  # channel-wise
    return sim.mean().item()

In [ ]:
print("Adversarial vs Random:", mean_cosine_similarity(fm1, fm2))

In [ ]:
(fm1-fm2).norm()

In [ ]:
fm1.sum()

## Training for random location

In [ ]:
!cp -r /kaggle/input/original-repo/adversarial-patch-transferability /kaggle/working/
!cp /kaggle/input/pidnet-s-weights/PIDNet_S_Cityscapes_test.pt /kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet
!cp /kaggle/input/pidnet-m-weights/PIDNet_M_Cityscapes_test.pt /kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet
!cp /kaggle/input/bisenetv2-weights/bisenetv2.pth /kaggle/working/adversarial-patch-transferability/pretrained_models/BisNetV2

In [ ]:
%cd /kaggle/working/adversarial-patch-transferability

In [ ]:
import yaml

yaml_path = "/kaggle/working/adversarial-patch-transferability/configs/config.yaml"

# Read
with open(yaml_path, 'r') as file:
    conf = yaml.safe_load(file)

print(conf)  # View content
conf["patch"]["loc"]="random"
with open(yaml_path, 'w') as file:
    yaml.safe_dump(conf, file)

In [ ]:
print(conf)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('/kaggle/working/adversarial-patch-transferability')
from utils.utils import setup_logging, get_config_from_yaml, process_config, print_config
from trainer.trainer import PatchTrainer
import torch
import pickle

In [ ]:
model_names = ['pidnet_s']
#model_names = ['icnet','bisenet_v1','bisenet_v2','segformer']
#model_names = ['segformer']
for model in model_names:
  ## getting the config file
  config = get_config_from_yaml('configs/config.yaml',model = model)
  #print_config(config)

  ## processing config to initialize directories for logs etc
  main_logger = process_config(config)

  ## training
  train_obj = PatchTrainer(config,main_logger)
  save = train_obj.train()
  pickle.dump( save, open(config.experiment.log_patch_address+config.model.name+"_randomloc"+".p", "wb" ) )

## BBFA experiment with modified loss on same layers

In [ ]:
!rm -rf /kaggle/working/adversarial-patch-transferability
!rm -rf /kaggle/working/state.db
!rm -rf /kaggle/working/logs
!rm -rf /kaggle/working/patch
%cd /kaggle/working

In [ ]:
!git clone https://github.com/SargamG/adversarial-patch-transferability

In [ ]:
!cp /kaggle/input/pidnet-s-weights/PIDNet_S_Cityscapes_test.pt /kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet
!cp /kaggle/input/pidnet-m-weights/PIDNet_M_Cityscapes_test.pt /kaggle/working/adversarial-patch-transferability/pretrained_models/PIDNet
!cp /kaggle/input/bisenetv2-weights/bisenetv2.pth /kaggle/working/adversarial-patch-transferability/pretrained_models/BisNetV2

In [ ]:
%cd /kaggle/working/adversarial-patch-transferability

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('/kaggle/working/adversarial-patch-transferability')
from utils.utils import setup_logging, get_config_from_yaml, process_config, print_config
from trainer.trainer import PatchTrainer
import torch
import pickle

In [ ]:
model = 'pidnet_s'
## getting the config file
config = get_config_from_yaml('configs/config.yaml',model = model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
## processing config to initialize directories for logs etc
main_logger = process_config(config)

## training
train_obj = PatchTrainer(config,main_logger,model)
train_obj.register_forward_hook1()
train_obj.register_forward_hook2()
H = torch.load('/kaggle/input/agg-gradient-for-pidnet-s-layer-3/H.pt', map_location = device)

In [ ]:
H

In [ ]:
save = train_obj.train(H)
pickle.dump( save, open(config.experiment.log_patch_address+config.model.name+"_bbfa_modifiedloss"+".p", "wb" ) )